In [1]:
%pip install nltk pandas
%pip install textstat
%pip install transformers torch scikit-learn
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/meitan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/meitan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
%pip install language-tool-python
import language_tool_python

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
df = pd.read_csv('nwp_feedback_7_25.csv')

# Structure

## Basic Lengths

In [4]:
# feedback word count
df['feedback'] = df['feedback'].astype(str)
df['wc'] = df['feedback'].apply(lambda x: len(x.split()))

In [5]:
# feedback sentence count
def count_sentences(feedback):
    sentences = sent_tokenize(feedback)
    return len(sentences)

df['sc'] = df['feedback'].apply(count_sentences)

## Complexity

In [6]:
# fleisch kincaid
import pandas as pd
import textstat
df['fk'] = df['feedback'].apply(textstat.flesch_reading_ease)

In [7]:
# TTR
import nltk
def calculate_ttr(feedback):
    tokens = word_tokenize(feedback)
    unique_tokens = set(tokens)
    ttr = len(unique_tokens) / len(tokens) if tokens else 0
    return ttr

df['ttr'] = df['feedback'].apply(calculate_ttr)

## Fluency

In [ ]:
# inverse perplexity
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Function to calculate inverse perplexity
def calculate_inverse_perplexity(feedback):
    inputs = tokenizer(feedback, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids'])
    loss = outputs.loss.item()
    perplexity = torch.exp(torch.tensor(loss)).item()
    inverse_perplexity = 1 / perplexity if perplexity != 0 else 0
    return inverse_perplexity

df['inverse_perplexity'] = df['feedback'].apply(calculate_inverse_perplexity)

In [9]:
# grammatical errors

# Initialize the language tool
tool = language_tool_python.LanguageTool('en-US')

# Function to count grammatical errors
def count_grammatical_errors(text):
    matches = tool.check(text)
    return len(matches)

df['gr_count'] = df['feedback'].apply(count_grammatical_errors)
df['gr_rate'] = df['gr_count'] / df['wc']

Unzipping /var/folders/y8/f8p3n2q93s17hqbx5z1j26000000gn/T/tmphlw63ale.zip to /Users/meitan/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /Users/meitan/.cache/language_tool_python.


In [10]:
df.head()

,tid,grade,prompt,essay,essayid,excerpt,feedback,startidx,endidx,commentid,...,gender,race,ed,ds,ell,sc,fk,ttr,gr_count,gr_rate
0,rfQv3LPBikeoFIE1NyA3,ms,Some of your friends perform community service...,"Dear Principal,\n\nI have heard you are having...",1CE6C10B9683,i,Capital and introductory phrase,255,258,1718398386391,...,F,White,0,0,0,1,-8.73,1.0,0,0.0
1,rfQv3LPBikeoFIE1NyA3,ms,Some of your friends perform community service...,"Dear Principal,\n\nI have heard you are having...",1CE6C10B9683,e a,Introductory phrase,548,551,1718398439880,...,F,White,0,0,0,1,-49.00,1.0,0,0.0
2,rfQv3LPBikeoFIE1NyA3,ms,Some of your friends perform community service...,"Dear Principal,\n\nI have heard you are having...",1CE6C10B9683,dyer,wrong word,1137,1141,1718398530582,...,F,White,0,0,0,1,120.21,1.0,1,0.5
3,rfQv3LPBikeoFIE1NyA3,ms,Some of your friends perform community service...,"Dear Principal,\n\nI have heard you are having...",1CE6C10B9683,know it may sound funny but picking up trash ...,run-on sentence,1267,1487,1718398574934,...,F,White,0,0,0,1,77.91,1.0,1,0.5
4,rfQv3LPBikeoFIE1NyA3,ms,Some of your friends perform community service...,"Dear Principal,\n\nI have heard you are having...",1CE6C10B9683,or example if one student did community servic...,Good example,791,875,1718398626223,...,F,White,0,0,0,1,35.61,1.0,0,0.0


# Specificity

## Content Word Density

In [11]:
import nltk
def calculate_content_word_density(feedback):
    tokens = word_tokenize(feedback)
    pos_tags = pos_tag(tokens)
    # Content words are nouns, verbs, adjectives, and adverbs
    content_words = [word for word, pos in pos_tags if pos.startswith('NN') or pos.startswith('VB') or pos.startswith('JJ') or pos.startswith('RB')]
    content_word_density = len(content_words) / len(tokens) if tokens else 0
    
    return content_word_density

df['content_word_density'] = df['feedback'].apply(calculate_content_word_density)

## Uptake

In [12]:
# def. uptake as overlap
def calculate_uptake(excerpt, feedback):
    # Tokenize both excerpt and feedback comment
    excerpt_tokens = set(word_tokenize(excerpt.lower()))
    feedback_tokens = set(word_tokenize(feedback.lower()))
    
    # Find common words
    common_words = excerpt_tokens.intersection(feedback_tokens)
    
    # Calculate uptake as the ratio of common words to total words in the excerpt
    uptake = len(common_words) / len(excerpt_tokens) if excerpt_tokens else 0
    
    return uptake

# Apply the function to calculate uptake for each row
df['uptake'] = df.apply(lambda row: calculate_uptake(row['excerpt'], row['feedback']), axis=1)

## Context Relevance

In [13]:
# cosine similarity of embeddings
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings

# Function to calculate cosine similarity between embeddings
def calculate_cosine_similarity(excerpt, feedback):
    excerpt_embedding = get_embeddings(excerpt)
    feedback_embedding = get_embeddings(feedback)
    similarity = cosine_similarity([excerpt_embedding], [feedback_embedding])[0][0]
    return similarity

# Apply the function to calculate cosine similarity for each row
df['cosine_similarity'] = df.apply(lambda row: calculate_cosine_similarity(row['excerpt'], row['feedback']), axis=1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Semantic Focus

## Pronouns

In [14]:
first_person_singular = {'i', 'me', 'my', 'mine'}
second_person = {'you', 'your', 'yours'}
first_person_plural = {'we', 'us', 'our', 'ours'}

def count_pronouns(feedback, pronoun_set):
    tokens = word_tokenize(feedback.lower())
    pronoun_count = sum(1 for token in tokens if token in pronoun_set)
    return pronoun_count

df['pronoun_fps'] = df['feedback'].apply(lambda x: count_pronouns(x, first_person_singular))
df['pronoun_sp'] = df['feedback'].apply(lambda x: count_pronouns(x, second_person))
df['pronoun_fpp'] = df['feedback'].apply(lambda x: count_pronouns(x, first_person_plural))


## Questions

In [15]:
wh_words = {'what', 'where', 'when', 'which', 'who', 'whom', 'whose', 'why', 'how'}
auxiliary_verbs = {'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'shall', 'should', 'can', 'could', 'may', 'might', 'must'}
helping_verbs = {"is", "am", "can", "are", "do", "does"}

def is_question(sentence):
    tokens = word_tokenize(sentence)
    return sentence.strip().endswith('?') or (tokens and (tokens[0] in wh_words or tokens[0] in helping_verbs))

def count_wh_questions(feedback):
    sentences = sent_tokenize(feedback.lower())
    wh_question_count = sum(1 for sentence in sentences if is_question(sentence) and word_tokenize(sentence)[0] in wh_words)
    return wh_question_count

def count_yes_no_questions(feedback):
    sentences = sent_tokenize(feedback.lower())
    yes_no_question_count = sum(1 for sentence in sentences if is_question(sentence) and word_tokenize(sentence)[0] in auxiliary_verbs)
    return yes_no_question_count

df['wh_question_count'] = df['feedback'].apply(count_wh_questions)
df['yes_no_question_count'] = df['feedback'].apply(count_yes_no_questions)


In [18]:
df.head()
df.to_csv('nwp_pt1.csv', index=False)

In [17]:
df.head()

,tid,grade,prompt,essay,essayid,excerpt,feedback,startidx,endidx,commentid,...,gr_count,gr_rate,content_word_density,uptake,cosine_similarity,pronoun_fps,pronoun_sp,pronoun_fpp,wh_question_count,yes_no_question_count
0,rfQv3LPBikeoFIE1NyA3,ms,Some of your friends perform community service...,"Dear Principal,\n\nI have heard you are having...",1CE6C10B9683,i,Capital and introductory phrase,255,258,1718398386391,...,0,0.0,0.75,0.0000,0.507323,0,0,0,0,0
1,rfQv3LPBikeoFIE1NyA3,ms,Some of your friends perform community service...,"Dear Principal,\n\nI have heard you are having...",1CE6C10B9683,e a,Introductory phrase,548,551,1718398439880,...,0,0.0,1.00,0.0000,0.654442,0,0,0,0,0
2,rfQv3LPBikeoFIE1NyA3,ms,Some of your friends perform community service...,"Dear Principal,\n\nI have heard you are having...",1CE6C10B9683,dyer,wrong word,1137,1141,1718398530582,...,1,0.5,1.00,0.0000,0.683321,0,0,0,0,0
3,rfQv3LPBikeoFIE1NyA3,ms,Some of your friends perform community service...,"Dear Principal,\n\nI have heard you are having...",1CE6C10B9683,know it may sound funny but picking up trash ...,run-on sentence,1267,1487,1718398574934,...,1,0.5,1.00,0.0000,0.426005,0,0,0,0,0
4,rfQv3LPBikeoFIE1NyA3,ms,Some of your friends perform community service...,"Dear Principal,\n\nI have heard you are having...",1CE6C10B9683,or example if one student did community servic...,Good example,791,875,1718398626223,...,0,0.0,1.00,0.0625,0.460753,0,0,0,0,0


In [16]:
import pandas as pd
from tqdm import tqdm
df = pd.read_csv('human_meta_pt1.csv')

In [96]:
df['feedback'] = df['feedback'].astype(str)
df['excerpt'] = df['excerpt'].astype(str)

## Non-Dialogic

In [97]:
from tqdm import tqdm
from transformers import pipeline
pipe = pipeline("text-classification", model="meiflwr/nondialogic_feedback", device = 0)
def get_dro(excerpt, feedback):
    out = []
    dro_out = pipe([{'text': feedback, 'text_pair': excerpt}])
    if dro_out[0]['label'] == 'LABEL_1':
        out.append(1)
    else:
        out.append(0)
    return out

tqdm.pandas()
df['ND'] = df.progress_apply(lambda row: get_dro(row['excerpt'], row['feedback']), axis=1)

/home/mxtan/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  0%|          | 0/385 [00:00<?, ?it/s]/home/mxtan/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 385/385 [00:02<00:00, 139.63it/s]


## Non-Revision-Oriented

In [98]:
from transformers import pipeline
pipe = pipeline("text-classification", model="meiflwr/nonrevisionoriented_feedback")
def get_dro(excerpt, feedback):
    out = []
    dro_out = pipe([{'text': feedback, 'text_pair': excerpt}])
    if dro_out[0]['label'] == 'LABEL_1':
        out.append(1)
    else:
        out.append(0)
    return out

tqdm.pandas()
df['NRO'] = df.progress_apply(lambda row: get_dro(row['excerpt'], row['feedback']), axis=1)

/home/mxtan/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 385/385 [00:11<00:00, 32.91it/s]


## Pure Praise

In [99]:
from transformers import pipeline
pipe = pipeline("text-classification", model="meiflwr/praise_feedback")
def get_dro(excerpt, feedback):
    out = []
    dro_out = pipe([{'text': feedback, 'text_pair': excerpt}])
    if dro_out[0]['label'] == 'LABEL_1':
        out.append(1)
    else:
        out.append(0)
    return out

tqdm.pandas()
df['PP'] = df.progress_apply(lambda row: get_dro(row['excerpt'], row['feedback']), axis=1)

/home/mxtan/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 385/385 [00:12<00:00, 30.64it/s]


## Power-Affirming

In [100]:
from tqdm import tqdm
# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-classification", model="meiflwr/power-affirming-feedback", device = 0)
def get_pa(excerpt, feedback):
  reward = pipe([{'text': feedback, 'text_pair': excerpt}])
  return reward[0]['score']

tqdm.pandas()
df['pa_mean'] = df.progress_apply(lambda row: get_pa(row['excerpt'], row['feedback']), axis=1)

/home/mxtan/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  0%|          | 0/385 [00:00<?, ?it/s]/home/mxtan/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 385/385 [00:02<00:00, 128.95it/s]


In [101]:
df['pa_mean'].mean()

0.5671552028748896

In [102]:
# Use a pipeline as a high-level helper
from transformers import pipeline
pipe1 = pipeline("text-classification", model="meiflwr/poweraffirming_feedback_skew1")
pipe2 = pipeline("text-classification", model="meiflwr/poweraffirming_feedbak_skew2")
pipe3 = pipeline("text-classification", model="meiflwr/poweraffirming_feedback_skew3")

def get_pa(excerpt, feedback, pipeline):
  reward = pipeline([{'text': feedback, 'text_pair': excerpt}])
  return reward[0]['score']

tqdm.pandas()
df['pa1'] = df.progress_apply(lambda row: get_pa(row['excerpt'], row['feedback'], pipe1), axis=1)
df['pa2'] = df.progress_apply(lambda row: get_pa(row['excerpt'], row['feedback'], pipe2), axis=1)
df['pa3'] = df.progress_apply(lambda row: get_pa(row['excerpt'], row['feedback'], pipe3), axis=1)

# Calculate the variance of the pa-scores
df['pa_var'] = df[['pa1', 'pa2', 'pa3']].var(axis=1)
df['pa_mean2'] = df[['pa1', 'pa2', 'pa3']].mean(axis=1)

/home/mxtan/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 385/385 [00:11<00:00, 33.14it/s]


In [103]:
print(df['pa1'].mean())
print(df['pa2'].mean())
print(df['pa3'].mean())
print(df['pa_mean2'].mean())

0.5651473980445366
0.5648816309191964
0.5653255993669684
0.565118209443567


# Save

In [104]:
df.to_csv('human_res_asap_meta.csv', index = False)